# Comparação de Modelos - Produção Solar França

Agrega métricas e gráficos dos modelos SARIMAX, MLP, Random Forest e Híbrido. Salva tabela comparativa (MSE, RMSE, MAE, MAPE, R²) em `out/solar_france/`.

In [ ]:
import pandas as pd
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize']=(12,6)
sns.set_palette('husl')

## 1. Carregar Métricas dos Modelos

In [ ]:
base = '../../out/solar_france'

# Paths dos arquivos de resultados
paths = {
    'SARIMAX': os.path.join(base, 'SARIMAX', 'sarimax_results.json'),
    'MLP': os.path.join(base, 'MLP', 'mlp_results.json'),
    'RandomForest': os.path.join(base, 'RandomForest', 'rf_results.json'),
    'Hibrido': os.path.join(base, 'Hibrido', 'hybrid_results.json')
}

# Coletar métricas
metrics = []
for name, p in paths.items():
    if os.path.exists(p):
        try:
            d = json.load(open(p))
            m = d.get('metrics', {})
            if m:
                metrics.append({
                    'modelo': name,
                    'mse': m.get('mse', np.nan),
                    'rmse': m.get('rmse', np.nan),
                    'mae': m.get('mae', np.nan),
                    'mape': m.get('mape', np.nan),
                    'r2': m.get('r2', np.nan)
                })
        except Exception as e:
            print(f'Erro ao ler {name}: {e}')
    else:
        print(f'Arquivo não encontrado: {p}')

# Criar DataFrame
dfm = pd.DataFrame(metrics)
print('\n' + '='*70)
print('COMPARAÇÃO DE MÉTRICAS - PRODUÇÃO SOLAR FRANÇA')
print('='*70)
print(dfm.to_string(index=False))
print('='*70)

## 2. Salvar Tabela Comparativa

In [ ]:
# Salvar CSV
comp_path = os.path.join(base, 'comparacao_metricas.csv')
dfm.to_csv(comp_path, index=False)
print(f'\n✓ Tabela salva em: {comp_path}')

## 3. Visualizações Comparativas

In [ ]:
# Gráficos de barras para cada métrica
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

metrics_to_plot = ['rmse', 'mae', 'mape', 'r2']
titles = ['RMSE (menor é melhor)', 'MAE (menor é melhor)', 
          'MAPE (menor é melhor)', 'R² (maior é melhor)']

for idx, (metric, title) in enumerate(zip(metrics_to_plot, titles)):
    ax = axes[idx // 2, idx % 2]
    if metric in dfm.columns:
        data_sorted = dfm.sort_values(metric, ascending=(metric != 'r2'))
        colors = ['#2ecc71' if i == 0 else '#3498db' for i in range(len(data_sorted))]
        if metric == 'r2':
            colors = ['#2ecc71' if i == len(data_sorted)-1 else '#3498db' 
                     for i in range(len(data_sorted))]
        
        ax.barh(data_sorted['modelo'], data_sorted[metric], color=colors, alpha=0.8)
        ax.set_title(title, fontsize=12, fontweight='bold')
        ax.set_xlabel(metric.upper())
        ax.grid(True, alpha=0.3, axis='x')
        
        # Adicionar valores nas barras
        for i, (model, value) in enumerate(zip(data_sorted['modelo'], data_sorted[metric])):
            ax.text(value, i, f' {value:.2f}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig(os.path.join(base, 'comparacao_metricas.png'), dpi=300, bbox_inches='tight')
plt.show()

print('\n✓ Gráficos salvos!')

## 4. Identificar Melhores Modelos

In [ ]:
if not dfm.empty:
    print('\n' + '='*70)
    print('RANKING DOS MODELOS')
    print('='*70)
    
    best_models = {}
    
    # Menores valores são melhores
    for metric in ['rmse', 'mae', 'mape']:
        if metric in dfm.columns:
            best_idx = dfm[metric].idxmin()
            best_model = dfm.loc[best_idx, 'modelo']
            best_value = dfm.loc[best_idx, metric]
            best_models[metric] = best_model
            print(f'Melhor {metric.upper()}: {best_model} ({best_value:.2f})')
    
    # Maior valor é melhor para R²
    if 'r2' in dfm.columns:
        best_idx = dfm['r2'].idxmax()
        best_model = dfm.loc[best_idx, 'modelo']
        best_value = dfm.loc[best_idx, 'r2']
        best_models['r2'] = best_model
        print(f'Melhor R²: {best_model} ({best_value:.4f})')
    
    print('='*70)
    
    # Modelo vencedor geral (mais frequente)
    from collections import Counter
    most_common = Counter(best_models.values()).most_common(1)
    if most_common:
        winner = most_common[0][0]
        count = most_common[0][1]
        print(f'\n🏆 MODELO VENCEDOR: {winner} (melhor em {count}/{len(best_models)} métricas)')
else:
    print('\n⚠️  Nenhuma métrica encontrada. Execute os modelos primeiro.')

## 5. Análise da Série Temporal

In [ ]:
# Carregar dados originais
raw_path = '../../data/solar_france.xlsx'

if os.path.exists(raw_path):
    df = pd.read_excel(raw_path)
    df['Date and Hour'] = pd.to_datetime(df['Date and Hour'])
    df = df.set_index('Date and Hour').sort_index().dropna()
    
    print('\n' + '='*70)
    print('CARACTERÍSTICAS DA SÉRIE TEMPORAL')
    print('='*70)
    print(f'Período: {df.index.min()} até {df.index.max()}')
    print(f'Total de observações: {len(df)}')
    print(f'Frequência: {pd.infer_freq(df.index)}')
    print(f'\nEstatísticas da Produção:')
    print(df['Production'].describe())
    
    # Decomposição sazonal
    try:
        dec = seasonal_decompose(df['Production'], model='additive', period=24)
        
        var_obs = np.var(dec.observed.dropna())
        var_sea = np.var(dec.seasonal.dropna())
        var_trd = np.var(dec.trend.dropna())
        var_res = np.var(dec.resid.dropna())
        
        seasonality_strength = max(0.0, 1 - var_res / (var_res + var_sea + 1e-9))
        trend_strength = max(0.0, 1 - var_res / (var_res + var_trd + 1e-9))
        
        print(f'\nForça da Sazonalidade (0-1): {seasonality_strength:.3f}')
        print(f'Força da Tendência (0-1): {trend_strength:.3f}')
        
        # Visualizar decomposição
        fig, axes = plt.subplots(4, 1, figsize=(15, 12))
        
        dec.observed.plot(ax=axes[0], title='Série Original')
        axes[0].set_ylabel('Produção')
        
        dec.trend.plot(ax=axes[1], title='Tendência')
        axes[1].set_ylabel('Tendência')
        
        dec.seasonal.plot(ax=axes[2], title='Componente Sazonal (24h)')
        axes[2].set_ylabel('Sazonalidade')
        
        dec.resid.plot(ax=axes[3], title='Resíduos')
        axes[3].set_ylabel('Resíduos')
        
        for ax in axes:
            ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(os.path.join(base, 'serie_decomposition.png'), dpi=300, bbox_inches='tight')
        plt.show()
        
    except Exception as e:
        print(f'\nErro na decomposição: {e}')
else:
    print(f'\n⚠️  Arquivo de dados não encontrado: {raw_path}')

## 6. Insights e Recomendações

In [ ]:
insights = []
insights.append('='*70)
insights.append('INSIGHTS E RECOMENDAÇÕES')
insights.append('='*70)
insights.append('')
insights.append('📊 Dataset: Produção Solar na França')
insights.append('   - Dados horários com forte padrão diário (ciclo dia/noite)')
insights.append('   - Sazonalidade clara de 24 horas')
insights.append('   - Produção zero durante a noite')
insights.append('')
insights.append('🔍 Características dos Modelos:')
insights.append('')
insights.append('1️⃣  SARIMAX (1,1,1)x(2,1,1,24):')
insights.append('   ✓ Captura bem componentes lineares e sazonalidade')
insights.append('   ✓ Interpretável e teoricamente fundamentado')
insights.append('   ✗ Pode ter dificuldade com não-linearidades')
insights.append('')
insights.append('2️⃣  MLP (Multi-Layer Perceptron):')
insights.append('   ✓ Captura padrões não-lineares complexos')
insights.append('   ✓ Bom com features de lag e temporais')
insights.append('   ✗ Requer mais dados e tuning')
insights.append('')
insights.append('3️⃣  Random Forest:')
insights.append('   ✓ Robusto e menos sensível a outliers')
insights.append('   ✓ Captura interações entre features')
insights.append('   ✗ Pode suavizar picos extremos')
insights.append('')
insights.append('4️⃣  Híbrido (SARIMAX + MLP):')
insights.append('   ✓ Combina o melhor dos dois mundos')
insights.append('   ✓ SARIMAX modela componente linear, MLP os resíduos')
insights.append('   ✓ Geralmente superior quando há sazonalidade + não-linearidade')
insights.append('')
insights.append('='*70)
insights.append('💡 RECOMENDAÇÕES:')
insights.append('='*70)

if not dfm.empty:
    # Recomendar com base nas métricas
    best_rmse = dfm.loc[dfm['rmse'].idxmin(), 'modelo'] if 'rmse' in dfm.columns else None
    best_r2 = dfm.loc[dfm['r2'].idxmax(), 'modelo'] if 'r2' in dfm.columns else None
    
    if best_rmse:
        insights.append(f'\n🎯 Para produção solar com sazonalidade diária forte:')
        insights.append(f'   Recomendado: {best_rmse}')
        insights.append(f'   Motivo: Melhor erro de previsão (RMSE)')
    
    if 'Hibrido' in dfm['modelo'].values:
        insights.append('\n🚀 O modelo Híbrido tende a ser superior porque:')
        insights.append('   - Captura a sazonalidade diária forte (SARIMAX)')
        insights.append('   - Modela padrões não-lineares residuais (MLP)')
        insights.append('   - Ideal para séries com ciclos claros + variabilidade')

insights.append('')
insights.append('='*70)

# Imprimir e salvar
for line in insights:
    print(line)

# Salvar resumo
with open(os.path.join(base, 'comparacao_resumo.txt'), 'w', encoding='utf-8') as f:
    f.write('\n'.join(insights))

print(f'\n✓ Resumo salvo em: {os.path.join(base, "comparacao_resumo.txt")}')
print('\n✅ Análise comparativa concluída!')